In [0]:
# this is alexnet from lab3

In [0]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [0]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [0]:
def valid_file(file_name):
  return not 'data.csv' in file_name

In [0]:
#,is_valid_file=valid_file_train

val_data = torchvision.datasets.ImageFolder('./drive/My Drive/Image_Data_Set/validation',loader = plt.imread,is_valid_file=valid_file,transform=transform)
test_data = torchvision.datasets.ImageFolder('./drive/My Drive/Image_Data_Set/test',loader = plt.imread,is_valid_file=valid_file,transform=transform)
train_data = torchvision.datasets.ImageFolder('./drive/My Drive/Image_Data_Set/train',loader = plt.imread,is_valid_file=valid_file,transform=transform)

total_data = len(val_data) + len(test_data) + len(train_data)

print("Validation set percentage: "+str(len(val_data)/total_data))
print("Test set percentage: "+str(len(test_data)/total_data))
print("Training set percentage: "+str(len(train_data)/total_data))

Validation set percentage: 0.15364161849710983
Test set percentage: 0.05164739884393064
Training set percentage: 0.7947109826589596


In [0]:
import torchvision.models
alexnet = torchvision.models.alexnet(pretrained=True).cuda()

In [0]:
def img_to_feature_val(data,batch_size):
  features = []
  label = []
  loader = torch.utils.data.DataLoader(data, batch_size=batch_size,shuffle=True)
  loader_size = len(loader)
  idx = 0
  for imgs, labels in iter(loader):
    idx += 1
    
    features.append(alexnet.features(imgs.cuda()))
    label.append(labels.cuda())
    if idx % 50==0:
      print ('Processing '+str(idx)+'/'+str(loader_size)+' ...')
      torch.save(features,'./drive/My Drive/val_bs4_pt/features_val_bs4_'+str(idx)+'.pt')
      torch.save(label,'./drive/My Drive/val_bs4_pt/labels_val_bs4_'+str(idx)+'.pt')
      print ('saved')
      del features
      del label
      torch.cuda.empty_cache()
      features = []
      label = []
  torch.save(features,'./drive/My Drive/val_bs4_pt/features_val_bs4_'+str(idx)+'.pt')
  torch.save(label,'./drive/My Drive/val_bs4_pt/labels_val_bs4_'+str(idx)+'.pt')
  print ('saved')
  del features
  del label
  torch.cuda.empty_cache()

In [0]:
img_to_feature_val(val_data,4)
#done

In [0]:
def img_to_feature_test(data,batch_size):
  features = []
  label = []
  loader = torch.utils.data.DataLoader(data, batch_size=batch_size,shuffle=True)
  loader_size = len(loader)
  idx = 0
  for imgs, labels in iter(loader):
    idx += 1
    
    features.append(alexnet.features(imgs.cuda()))
    label.append(labels.cuda())
    if idx % 50==0:
      print ('Processing '+str(idx)+'/'+str(loader_size)+' ...')
      torch.save(features,'./drive/My Drive/test_bs4_pt/features_test_bs4_'+str(idx)+'.pt')
      torch.save(label,'./drive/My Drive/test_bs4_pt/labels_test_bs4_'+str(idx)+'.pt')
      print ('saved')
      del features
      del label
      torch.cuda.empty_cache()
      features = []
      label = []
  torch.save(features,'./drive/My Drive/test_bs4_pt/features_test_bs4_'+str(idx)+'.pt')
  torch.save(label,'./drive/My Drive/test_bs4_pt/labels_test_bs4_'+str(idx)+'.pt')
  print ('saved')
  del features
  del label
  torch.cuda.empty_cache()

In [0]:
img_to_feature_test(test_data,4)
#done

In [0]:
def img_to_feature_train(data,batch_size):
  features = []
  label = []
  loader = torch.utils.data.DataLoader(data, batch_size=batch_size,shuffle=True)
  loader_size = len(loader)
  idx = 0
  for imgs, labels in iter(loader):
    idx += 1
    
    features.append(alexnet.features(imgs.cuda()))
    label.append(labels.cuda())
    if idx % 50==0:
      print ('Processing '+str(idx)+'/'+str(loader_size)+' ...')
      torch.save(features,'./drive/My Drive/train_bs4_pt/features_train_bs4_'+str(idx)+'.pt')
      torch.save(label,'./drive/My Drive/train_bs4_pt/labels_train_bs4_'+str(idx)+'.pt')
      print ('saved')
      del features
      del label
      torch.cuda.empty_cache()
      features = []
      label = []
  torch.save(features,'./drive/My Drive/train_bs4_pt/features_train_bs4_'+str(idx)+'.pt')
  torch.save(label,'./drive/My Drive/train_bs4_pt/labels_train_bs4_'+str(idx)+'.pt')
  print ('saved')
  del features
  del label
  torch.cuda.empty_cache()

In [0]:
img_to_feature_train(train_data,4)

In [0]:
#combine val pt

val_idx = list(range(50,1329,50))+[1329]
val_total_features = []
val_total_labels = []
for i in val_idx:
  features = torch.load('./drive/My Drive/val_bs4_pt/features_val_bs4_'+str(i)+'.pt')
  labels = torch.load('./drive/My Drive/val_bs4_pt/labels_val_bs4_'+str(i)+'.pt')  
  print ('processing '+str(i)+'/1329 ...')
  for j in range(len(features)):
    val_total_features.append(features[j])
    val_total_labels.append(labels[j])
torch.save(val_total_features,'./drive/My Drive/combined_pt/val_features.pt')
torch.save(val_total_labels,'./drive/My Drive/combined_pt/val_labels.pt')
del val_total_features
del val_total_labels
del features
del labels 

In [0]:
#combine test pt

test_idx = list(range(50,447,50))+[447]
test_total_features = []
test_total_labels = []
for i in test_idx:
  features = torch.load('./drive/My Drive/test_bs4_pt/features_test_bs4_'+str(i)+'.pt')
  labels = torch.load('./drive/My Drive/test_bs4_pt/labels_test_bs4_'+str(i)+'.pt')  
  print ('processing '+str(i)+'/447 ...')
  for j in range(len(features)):
    test_total_features.append(features[j])
    test_total_labels.append(labels[j])
torch.save(test_total_features,'./drive/My Drive/combined_pt/test_features.pt')
torch.save(test_total_labels,'./drive/My Drive/combined_pt/test_labels.pt')
del test_total_features
del test_total_labels
del features
del labels 

In [0]:
#combine train pt

train_idx = list(range(5000,6875,50))+[6875]
train_total_features = []
train_total_labels = []
for i in train_idx:
  features = torch.load('./drive/My Drive/train_bs4_pt/features_train_bs4_'+str(i)+'.pt')
  labels = torch.load('./drive/My Drive/train_bs4_pt/labels_train_bs4_'+str(i)+'.pt')
  print ('processing '+str(i)+'/6875 ...')
  for j in range(len(features)):
    train_total_features.append(features[j])
    train_total_labels.append(labels[j])
torch.save(train_total_features,'./drive/My Drive/combined_pt/train_features_5.pt')
torch.save(train_total_labels,'./drive/My Drive/combined_pt/train_labels_5.pt')
del train_total_features
del train_total_labels
del features
del labels

In [0]:
class CNN_transfer(nn.Module):
    def __init__(self):
        super(CNN_transfer, self).__init__()
        self.name = "net_transfer"
        

        self.fc1 = nn.Linear(256*19*19, 100)
        self.fc2 = nn.Linear(100, 32)
        
        
    def forward(self, x):

        
        x = x.view(-1, 256*19*19)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        #x = x.squeeze(1) # Flatten to [batch_size]
        return x

In [0]:
def get_train_accuracy_transfer(model,batch_size=4):
    
    train_idx = list(range(6))        

    correct = 0
    total = 0
    for j in train_idx:
      data = torch.load('./drive/My Drive/combined_pt/train_features_'+str(j)+'.pt')
      label = torch.load('./drive/My Drive/combined_pt/train_labels_'+str(j)+'.pt')
      for i in range(0,len(data)):
          model = model.cuda()
          output = model(data[i].cuda()).cuda() # We don't need to run F.softmax
          pred = output.max(1, keepdim=True)[1].cuda() # get the index of the max log-probability
          correct += pred.eq(label[i].view_as(pred)).sum().item()
          total += data[i].shape[0]
      del data
      del label
    
    return correct / total

In [0]:
def get_val_accuracy_transfer(model,batch_size=4):  

    correct = 0
    total = 0
    
    data = torch.load('./drive/My Drive/combined_pt/val_features.pt')
    label = torch.load('./drive/My Drive/combined_pt/val_labels.pt')
    for i in range(0,len(data)):
        model = model.cuda()
        output = model(data[i].cuda()).cuda() # We don't need to run F.softmax
        pred = output.max(1, keepdim=True)[1].cuda() # get the index of the max log-probability
        correct += pred.eq(label[i].view_as(pred)).sum().item()
        total += data[i].shape[0]
    del data
    del label
    return correct / total

In [0]:
def get_val_loss(model,batch_size=4):  

  data = torch.load('./drive/My Drive/combined_pt/val_features.pt')
  label = torch.load('./drive/My Drive/combined_pt/val_labels.pt')
  losses = []
  criterion = nn.CrossEntropyLoss().cuda()
  for i in range(0,len(data)):
      model = model.cuda()
      output = model(data[i].cuda()).cuda() # We don't need to run F.softmax
      loss = criterion(output, label[i].cuda())
      losses.append(float(loss)/batch_size)
  del data
  del label
  return sum(losses)/len(losses)

In [0]:
def train_transfer(model, batch_size=4, num_epochs=100, learning_rate = 0.0006,weight_decay=0.0002):
  
    
    start_time = time.time()
    
    torch.manual_seed(1000)

    train_idx = list(range(6))
    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay = weight_decay)
    

    iters, train_losses, val_losses, train_acc, val_acc = [], [], [], [], []

    # training
    n = 0 # the number of iterations
    x_value = 0
    for epoch in range(num_epochs):
        for j in train_idx: 
          train_loader = torch.load('./drive/My Drive/combined_pt/train_features_'+str(j)+'.pt')
          label_loader = torch.load('./drive/My Drive/combined_pt/train_labels_'+str(j)+'.pt')
          for i in range(0,len(train_loader)):

              out = model(train_loader[i].cuda()).cuda()             # forward pass
              loss = criterion(out, label_loader[i].cuda()) # compute the total loss


              loss.backward()               # backward pass (compute parameter updates)
              optimizer.step()              # make the updates for each parameter
              optimizer.zero_grad()         # a clean up step for PyTorch

              # save the current training information
        iters.append(n)
        train_losses.append(float(loss)/batch_size)             # compute *average* loss
        val_losses.append(get_val_loss(model,batch_size))
        n += 1
        del train_loader
        del label_loader
        train_acc.append(get_train_accuracy_transfer(model,batch_size=batch_size)) # compute training accuracy 
        val_acc.append(get_val_accuracy_transfer(model,batch_size=batch_size))  # compute validation accuracy


        print(("Epoch {}: Train acc: {} |"+"Validation acc: {}").format(epoch + 101,train_acc[-1],val_acc[-1]))
        model_path = "bs{0}_lr{1}_epoch{2}".format(batch_size,learning_rate,epoch+100)
        torch.save(model.state_dict(), './drive/My Drive/baseline_model_saved/'+model_path+'.pt')

    # plotting
    plt.title("Training Curve")
    plt.plot(iters, train_losses, label="Train")
    plt.plot(iters, val_losses, label='Validation')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()

    plt.title("Training Curve")
    plt.plot(iters, train_acc, label="Train")
    plt.plot(iters, val_acc, label="Validation")
    plt.xlabel("Epochs")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()
    
    end_time = time.time()
    duration = end_time - start_time

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))
    print ("Trained Duration: {} seconds".format(duration))

In [0]:
after_epoch100 = CNN_transfer().cuda()
state = torch.load('./drive/My Drive/baseline_model_saved/bs4_lr0.0006_epoch99.pt')
after_epoch100.load_state_dict(state)
train_transfer(after_epoch100)

In [0]:
after_alexnet = CNN_transfer().cuda()
train_transfer(after_alexnet)

In [0]:
after_alexnet_v2 = CNN_transfer().cuda()
train_transfer(after_alexnet_v2,learning_rate=0.0004,weight_decay=0.0001)

In [0]:
def get_test_accuracy_transfer(model,batch_size=4):
   
    data = torch.load('features_test_bs4.pt')
    label = torch.load('label_test_bs4.pt')
    
    correct = 0
    total = 0
    for i in range(0,len(data)):
        output = model(data[i]) # We don't need to run F.softmax
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(label[i].view_as(pred)).sum().item()
        total += data[i].shape[0]
    return correct / total